# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 10:29AM,252815,19,N17874,"NAPP Technologies, LLC",Released
1,Dec 12 2022 10:29AM,252814,10,MSP216649,"Methapharm, Inc.",Released
2,Dec 12 2022 10:29AM,252814,10,MSP216651,"Methapharm, Inc.",Released
3,Dec 12 2022 10:29AM,252814,10,MSP216658,Methapharm-G,Released
4,Dec 12 2022 10:29AM,252814,10,MSP216614,"Methapharm, Inc.",Released
5,Dec 12 2022 10:28AM,252813,10,MSP216655,Methapharm-G,Released
6,Dec 12 2022 10:26AM,252812,10,PRF-44319,Bio-PRF,Released
7,Dec 12 2022 10:26AM,252812,10,PRF-44361,Bio-PRF,Released
8,Dec 12 2022 10:26AM,252812,10,PRF-44364,Bio-PRF,Released
9,Dec 12 2022 10:26AM,252812,10,PRF-44367,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,252811,Released,1
22,252812,Released,18
23,252813,Released,1
24,252814,Released,4
25,252815,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252811,NaN,1.0
252812,NaN,18.0
252813,NaN,1.0
252814,NaN,4.0
252815,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252777,0.0,1.0
252778,0.0,1.0
252779,0.0,1.0
252783,0.0,1.0
252784,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252777,0,1
252778,0,1
252779,0,1
252783,0,1
252784,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252777,0,1
1,252778,0,1
2,252779,0,1
3,252783,0,1
4,252784,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252777,,1
1,252778,,1
2,252779,,1
3,252783,,1
4,252784,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC"
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc."
3,Dec 12 2022 10:29AM,252814,10,Methapharm-G
5,Dec 12 2022 10:28AM,252813,10,Methapharm-G
6,Dec 12 2022 10:26AM,252812,10,Bio-PRF
24,Dec 12 2022 10:25AM,252810,10,Bio-PRF
30,Dec 12 2022 10:08AM,252811,10,"Senseonics, Incorporated"
31,Dec 12 2022 10:02AM,252807,19,"AdvaGen Pharma, Ltd"
33,Dec 12 2022 9:45AM,252801,10,ISDIN Corporation
42,Dec 12 2022 9:44AM,252802,12,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",,1
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc.",,4
2,Dec 12 2022 10:29AM,252814,10,Methapharm-G,,4
3,Dec 12 2022 10:28AM,252813,10,Methapharm-G,,1
4,Dec 12 2022 10:26AM,252812,10,Bio-PRF,,18
5,Dec 12 2022 10:25AM,252810,10,Bio-PRF,,6
6,Dec 12 2022 10:08AM,252811,10,"Senseonics, Incorporated",,1
7,Dec 12 2022 10:02AM,252807,19,"AdvaGen Pharma, Ltd",1,1
8,Dec 12 2022 9:45AM,252801,10,ISDIN Corporation,,9
9,Dec 12 2022 9:44AM,252802,12,SugarBear,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",1,
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc.",4,
2,Dec 12 2022 10:29AM,252814,10,Methapharm-G,4,
3,Dec 12 2022 10:28AM,252813,10,Methapharm-G,1,
4,Dec 12 2022 10:26AM,252812,10,Bio-PRF,18,
5,Dec 12 2022 10:25AM,252810,10,Bio-PRF,6,
6,Dec 12 2022 10:08AM,252811,10,"Senseonics, Incorporated",1,
7,Dec 12 2022 10:02AM,252807,19,"AdvaGen Pharma, Ltd",1,1
8,Dec 12 2022 9:45AM,252801,10,ISDIN Corporation,9,
9,Dec 12 2022 9:44AM,252802,12,SugarBear,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",1,
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc.",4,
2,Dec 12 2022 10:29AM,252814,10,Methapharm-G,4,
3,Dec 12 2022 10:28AM,252813,10,Methapharm-G,1,
4,Dec 12 2022 10:26AM,252812,10,Bio-PRF,18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",1,NaN
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc.",4,NaN
2,Dec 12 2022 10:29AM,252814,10,Methapharm-G,4,NaN
3,Dec 12 2022 10:28AM,252813,10,Methapharm-G,1,NaN
4,Dec 12 2022 10:26AM,252812,10,Bio-PRF,18,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 10:29AM,252815,19,"NAPP Technologies, LLC",1,0.0
1,Dec 12 2022 10:29AM,252814,10,"Methapharm, Inc.",4,0.0
2,Dec 12 2022 10:29AM,252814,10,Methapharm-G,4,0.0
3,Dec 12 2022 10:28AM,252813,10,Methapharm-G,1,0.0
4,Dec 12 2022 10:26AM,252812,10,Bio-PRF,18,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3792027,270,4.0
12,758374,3,0.0
19,758406,3,1.0
20,758341,3,0.0
22,252779,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3792027,270,4.0
1,12,758374,3,0.0
2,19,758406,3,1.0
3,20,758341,3,0.0
4,22,252779,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,270,4.0
1,12,3,0.0
2,19,3,1.0
3,20,3,0.0
4,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,270.0
1,12,Released,3.0
2,19,Released,3.0
3,20,Released,3.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,4.0,0.0,1.0,0.0,0.0
Released,270.0,3.0,3.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,4.0,0.0,1.0,0.0,0.0
1,Released,270.0,3.0,3.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,4.0,0.0,1.0,0.0,0.0
1,Released,270.0,3.0,3.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()